In [1]:
import requests
import psycopg2
import psycopg2.extras as pgex
import pandas as pd
import numpy as np
import time
import logging
from datetime import datetime

In [2]:
t = time.time()
currentDate = time.strftime('%Y-%m-%d', time.localtime(t))

In [3]:
###logging.basicConfig(filename="scriptLog.log", filemode='w', force="true", level=logging.INFO)

In [4]:
logger = logging.getLogger('shazamablam')
logger.setLevel(logging.INFO)

fh = logging.FileHandler('scriptLog.log')
fh.setLevel(logging.INFO)

logger.addHandler(fh)

In [5]:
logger.info(f'{currentDate}: SCRIPT START')

In [6]:
import configparser

config = configparser.ConfigParser()
config.read('./database.ini')
configHost = config.get('postgresql', 'host')
configPort = config.get('postgresql', 'port')
configDatabase = config.get('postgresql', 'database')
configUser = config.get('postgresql', 'user')
configPassword = config.get('postgresql', 'password')

In [7]:
def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)

#register_adapter(np.int64, addapt_numpy_int64)

class Client:
    
    def __init__(self):
        
        self.conn = None
        self.cur = None
        
    def connect(self):
        self.conn = psycopg2.connect(
            host=configHost,
            port=configPort,
            database=configDatabase,
            user=configUser,
            password=configPassword
        )
        self.cur = self.conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    def disconnect(self):
        if self.cur is not None:
            self.cur.close()
            self.cur = None
        if self.conn is not None:
            self.conn.close()
            self.conn = None
    def reset(self):
        self.disconnect()
        self.connect()
    def getCurrentColumns(self):
        return [i[0] for i in self.cur.description]       
    def getDfFromCur(self):
        cols = self.getCurrentColumns()
        data = self.cur.fetchall()
        return pd.DataFrame(data, columns=cols)

In [8]:
# Utility Functions
def dfColToTuple(dataframe):
    return tuple(list(dataframe.columns))

def df2List(dataframe): 
    return list(dataframe.itertuples(index=False, name=None))

def removeDuplicatesForInsert(newDF, oldDF, colName):
    ids = oldDF[colName].values
    return newDF[~newDF[colName].isin(ids)].reset_index(drop=True)

def getDuplicatesForInsert(newDF, oldDF, colName):
    ids = oldDF[colName].values
    return newDF[newDF[colName].isin(ids)].reset_index(drop=True)

def tryToGetByKey(dictionary, key):
    if (key in dictionary):
        return dictionary[key]

In [9]:
client = Client()
client.connect()

In [10]:
citiesDF = pd.read_csv('./cities.csv')
cityArr = citiesDF['city_id'].values

In [11]:
songArr = []
artistSongRefArr = []
artistArr = []

In [12]:
# loop through cityID and make requests
logger.info(f'########### SCRAPE START ###########')

for city_index, city_id in enumerate(cityArr): 
    requestString = f'https://www.shazam.com/shazam/v3/en-US/US/web/-/tracks/ip-city-chart-{city_id}?pageSize=200&startFrom=0'
    response = requests.get(requestString)
    data = response.json()
    print(f'City: {city_index + 1}/202 processed.')
    logger.info(f'City: {city_index + 1}/202 processed.')
    
    for songIndex, song in enumerate(data['tracks']):
        background = ""
        coverart = ""
        coverarthq = ""
        
        if 'images' in song:        
            background = tryToGetByKey(song['images'], 'background')
            coverart = tryToGetByKey(song['images'], 'coverart')
            coverarthq = tryToGetByKey(song['images'], 'coverarthq')
              
        songToInsert = {
            'song_id': song['key'], 
            'song_name': song['title'],
            'subtitle': song['subtitle'],
            'shazam_url': song['url'],
            'background': background,
            'coverart': coverart,
            'coverarthq': coverarthq,
            'rank': songIndex + 1,
            'city_id': city_id,
            'record_date': currentDate
        }
        
        songArr.append(songToInsert)
        
        if (song.get('artists') is not None): 
            for artist in song['artists']:
                refToInsert = {
                    'artist_id': artist['id'],
                    'song_id': song['key']
                }
                artistToInsert = {
                    'artist_id': artist['id'],
                    'artist_name': artist['alias']
                }
                artistSongRefArr.append(refToInsert)
                artistArr.append(artistToInsert)
                
    time.sleep(1)

City: 1/202 processed.
City: 2/202 processed.
City: 3/202 processed.
City: 4/202 processed.
City: 5/202 processed.


KeyboardInterrupt: 

In [ ]:
# Data frames with duplicates

songDF = pd.DataFrame(songArr)
artistSongRefDF = pd.DataFrame(artistSongRefArr)
artistDF = pd.DataFrame(artistArr)

In [ ]:
# Create Records DF

recordsDF = songDF[['song_id', 'rank', 'city_id', 'record_date']]
recordsDFnoDup = recordsDF.drop_duplicates(subset=['song_id', 'city_id']).reset_index(drop=True)

In [ ]:
# Song DF to be inserted

parsedSongDF = songDF[['song_id', 'song_name', 'subtitle', 'background', 'coverart', 'coverarthq', 'shazam_url']]
parsedSongDFnoDup = parsedSongDF.drop_duplicates(subset=['song_id']).reset_index(drop=True)

In [ ]:
# Artist DF to be inserted

artistDFnoDup = artistDF.drop_duplicates(subset=['artist_id']).reset_index(drop=True)

In [ ]:
# ArtistSongRefDF to be inserted

artistSongRefDFnoDup = artistSongRefDF.drop_duplicates(subset=['artist_id', 'song_id']).reset_index(drop=True)

In [ ]:
# Song insert query
songQuery = """
    insert into shazamablam.song
    ( 
        song_id, 
        song_name, 
        subtitle, 
        background, 
        coverart,
        coverarthq,
        shazam_url
    ) 
    values %s
"""

In [ ]:
# City insert query
cityQuery = """
    insert into shazamablam.city
    (
        city_id, 
        city_name,
        state_id
    )
    values %s
"""

In [ ]:
artistQuery = """
    insert into shazamablam.artist
    ( 
        artist_id, 
        artist_name
    ) 
    values %s
"""

In [ ]:
artistSongRefQuery = """
    insert into shazamablam.artist_song_ref
    (
        artist_id,
        song_id
    ) 
    values %s
"""

In [ ]:
recordsQuery = """
    insert into shazamablam.record
    (
        song_id,
        song_rank,
        city_id
    ) 
    values %s
"""

In [ ]:
# Insert city into database

client.cur.execute('select city_id from shazamablam.city')
dbCities = client.getDfFromCur()

citiesDF['city_id']  = citiesDF['city_id'].astype(str)

cityPackage = removeDuplicatesForInsert(citiesDF, dbCities, 'city_id').reset_index(drop=True)

pgex.execute_values(client.cur, cityQuery, df2List(cityPackage))

In [ ]:
# Insert song into database accounting for duplicates

client.cur.execute('select song_id from shazamablam.song')
dbSongs = client.getDfFromCur()

songPackage = removeDuplicatesForInsert(parsedSongDFnoDup, dbSongs, 'song_id').reset_index(drop=True)

In [ ]:
# Insert artist into database accounting for duplicates

client.cur.execute('select artist_id from shazamablam.artist')
dbArtists = client.getDfFromCur()

artistPackage = removeDuplicatesForInsert(artistDFnoDup, dbArtists, 'artist_id').reset_index(drop=True)

pgex.execute_values(client.cur, artistQuery, df2List(artistPackage))

In [ ]:
# Insert artist_song_ref 

refPackage = getDuplicatesForInsert(artistSongRefDFnoDup, songPackage, 'song_id').reset_index(drop=True)
pgex.execute_values(client.cur, songQuery, df2List(songPackage))

In [ ]:
# Perform insertion query 

pgex.execute_values(client.cur, artistSongRefQuery, df2List(refPackage.applymap(str)))

In [ ]:
# TODO Insert record into database

client.cur.execute('select record_date from shazamablam.record')
dbRecords = client.getDfFromCur()

dbRecords['record_date']  = dbRecords['record_date'].astype(str)
recordsDFnoDup['record_date']  = recordsDFnoDup['record_date'].astype(str)

recPackage = removeDuplicatesForInsert(recordsDFnoDup, dbRecords, 'record_date')

pgex.execute_values(client.cur, recordsQuery, df2List(recPackage[['song_id', 'rank', 'city_id']]))

In [ ]:
client.conn.commit()
client.disconnect()

In [ ]:
logger.info(f'{currentDate}: SCRIPT END ')